In [1]:
import numpy as np
import pandas as pd
S0 = np.loadtxt('S0.txt')
S0 = np.array(S0)
S0

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [2]:
STS = np.dot(S0.T, S0)
STS

array([[3454.5 ,  896.5 ,  430.75, ...,    0.  ,    0.  ,    0.  ],
       [ 896.5 , 1380.25,  259.  , ...,    0.  ,    0.  ,    0.  ],
       [ 430.75,  259.  ,  609.25, ...,    0.  ,    0.  ,    0.  ],
       ...,
       [   0.  ,    0.  ,    0.  , ...,   12.25,   12.25,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,   12.25,   12.25,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,   16.  ]])

In [3]:
def power_iteration_eigen(matrix, num_iterations=1000, tolerance=1e-10):
    n, _ = matrix.shape
    x = np.random.rand(n)
    x = x / np.linalg.norm(x)
    for _ in range(num_iterations):
        x_new = np.dot(matrix, x)
        eigenvalue = np.dot(x, x_new)
        x_new = x_new / np.linalg.norm(x_new)
        if np.linalg.norm(x - x_new) < tolerance:
            break
        x = x_new
    return eigenvalue, x

def eigenvalues_and_eigenvectors(matrix, num_iterations=10, tolerance=1e-3):
    eigenvalues = []
    eigenvectors = []
    for _ in range(40):
        eigenvalue, eigenvector = power_iteration_eigen(matrix, num_iterations, tolerance)
        if eigenvalue > 1:
            eigenvalues.append(eigenvalue)
            eigenvectors.append(eigenvector)
            matrix = matrix - eigenvalue * np.outer(eigenvector, eigenvector)
    return eigenvalues, eigenvectors

def svd(matrix, covariance_matrix):
    eigenvalues, eigenvectors = eigenvalues_and_eigenvectors(covariance_matrix)
    eigenvalues = np.array(eigenvalues)
    eigenvectors = np.array(eigenvectors)
    eigenvalues = np.sort(eigenvalues)[::-1]
    singular_values = np.sqrt(eigenvalues)
    right_singular_vectors = eigenvectors
    left_singular_vectors = np.dot(right_singular_vectors, matrix.T).T / singular_values
    return left_singular_vectors, np.diag(singular_values), right_singular_vectors
U, S, VT = svd(S0, STS)

In [38]:
def cosine_similarity(vec1, vec2):
    magnitude = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    if magnitude == 0:
        return 0  # Avoid division by zero
    return np.dot(vec1, vec2) / magnitude

def recommend_movies(user_id, compressed_matrix, ratings_matrix):
    user_ratings = ratings_matrix[user_id]
    similarities = []
    for i in range(len(ratings_matrix)):
        if i != user_id:
            similarity = cosine_similarity(user_ratings, ratings_matrix[i])
            similarities.append((i, similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)  # Sort by decreasing similarity
    
    recommended_movies = []
    for movie_index in range(len(ratings_matrix[user_id])):
        if user_ratings[movie_index] == 0:
            score = 0
            for similar_user_id, similarity in similarities:
                score += similarity * ratings_matrix[similar_user_id][movie_index]
            recommended_movies.append((movie_index, score))

    recommended_movies.sort(key=lambda x: x[1], reverse=True)  # Sort by decreasing score

    return recommended_movies, similarities


user_id = input('enter the user id')
number = input('enter the number of movies you want to be recommended')
number = int(number)
user_id = int(user_id)
recommended_movies, similarities = recommend_movies(user_id, user_ratings_est, S0)
df = pd.read_csv('movies.csv')

for tuple in recommended_movies[:number]:
    print(df.at[tuple[0], 'title'])


Shawshank Redemption, The (1994)
Star Wars: Episode IV - A New Hope (1977)
Silence of the Lambs, The (1991)
Star Wars: Episode V - The Empire Strikes Back (1980)
Inception (2010)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Schindler's List (1993)
Memento (2000)
